In [ ]:

import pandas as pd

# 1️⃣ Load your downloaded file
path = "/content/loan_purposes_31_state_MA.csv"
df = pd.read_csv(path, low_memory=False)

print("✅ File loaded")
print("Rows:", len(df))
print("Columns:", len(df.columns))
print("\nColumn names:")
print(df.columns.tolist())

# 2️⃣ Filter to 2023 if the column exists
if "activity_year" in df.columns:
    before = len(df)
    df = df[df["activity_year"] == 2023]
    print(f"\nFiltered to 2023 using 'activity_year': {before} → {len(df)} rows")

# 3️⃣ Sanity check for Massachusetts columns
if "state_code" in df.columns:
    print("\nState code value counts (top 5):")
    print(df["state_code"].value_counts().head())

if "state_name" in df.columns:
    print("\nState name value counts (top 5):")
    print(df["state_name"].value_counts().head())

# 4️⃣ Preview a few rows
print("\nSample rows:")
print(df.head())

# 5️⃣ Save a clean version for Power BI
output_path = "/content/MA_2023_Massachusetts_loans_clean.csv"
df.to_csv(output_path, index=False)
print(f"\n💾 Clean file saved to: {output_path}")


✅ File loaded
Rows: 22554
Columns: 99

Column names:
['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open-end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'multifamily_affordable_units', 'inc

In [ ]:
import pandas as pd

df = pd.read_csv("/content/MA_2023_Massachusetts_loans_clean.csv", low_memory=False)

columns_to_keep = [
    'activity_year','state_code','county_code','census_tract',
    'loan_type','loan_purpose','loan_amount','loan_to_value_ratio',
    'interest_rate','rate_spread','property_value','income',
    'debt_to_income_ratio','applicant_age','applicant_credit_score_type',
    'derived_dwelling_category','conforming_loan_limit','action_taken'
]

clean_df = df[columns_to_keep]

clean_df.to_csv("/content/MA_2023_dashboard_ready.csv", index=False)

print("Saved: /content/MA_2023_dashboard_ready.csv")
print("Rows:", len(clean_df))
print("Columns:", len(clean_df.columns))


Saved: /content/MA_2023_dashboard_ready.csv
Rows: 22554
Columns: 18


In [ ]:
clean_df.head()

,activity_year,state_code,county_code,census_tract,loan_type,loan_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,property_value,income,debt_to_income_ratio,applicant_age,applicant_credit_score_type,derived_dwelling_category,conforming_loan_limit,action_taken
0,2023,MA,25009.0,2.500921e+10,1,31,115000.0,35.48400,7.125,0.622,315000,40.0,46,35-44,1,Single Family (1-4 Units):Site-Built,C,1
1,2023,MA,25023.0,2.502353e+10,1,31,385000.0,NaN,NaN,NaN,NaN,106.0,NaN,45-54,9,Single Family (1-4 Units):Site-Built,C,4
2,2023,MA,25017.0,2.501731e+10,2,31,515000.0,97.68900,7.250,1.240,515000,163.0,43,<25,2,Single Family (1-4 Units):Site-Built,C,1
3,2023,MA,25017.0,2.501733e+10,1,31,495000.0,69.01400,6.325,-0.134,715000,122.0,45,35-44,2,Single Family (1-4 Units):Site-Built,C,1
4,2023,MA,25009.0,2.500925e+10,1,31,335000.0,80.96400,7.625,1.391,415000,178.0,30%-<36%,55-64,3,Single Family (1-4 Units):Site-Built,C,1


In [ ]:
import pandas as pd

# 1️⃣ Load your base file
df = pd.read_csv("/content/MA_2023_Massachusetts_loans_clean.csv", low_memory=False)

# ===============================
# 2️⃣ LOAN TYPE → LABEL
# ===============================
loan_type_map = {
    1: "Conventional",
    2: "FHA-insured",
    3: "VA-guaranteed",
    4: "USDA (RHS/FSA)"
}

df["loan_type_label"] = df["loan_type"].map(loan_type_map)

# ===============================
# 3️⃣ LOAN PURPOSE → LABEL
# ===============================
loan_purpose_map = {
    1: "Home Purchase",
    2: "Home Improvement",
    31: "Refinancing",
    32: "Cash-out Refinancing",
    4: "Other Purpose",
    5: "Not Applicable"
}

df["loan_purpose_label"] = df["loan_purpose"].map(loan_purpose_map)

# ===============================
# 4️⃣ ACTION TAKEN → LABEL + CATEGORY
# ===============================
action_map = {
    1: "Approved - Originated",
    2: "Approved - Not Accepted",
    3: "Denied",
    4: "Withdrawn",
    5: "Incomplete",
    6: "Purchased Loan",
    7: "Preapproval Denied",
    8: "Preapproval Approved - Not Accepted"
}

df["action_taken_label"] = df["action_taken"].map(action_map)

def categorize_action(val):
    if val in [1, 2]:
        return "Approved"
    elif val in [3, 7]:
        return "Denied"
    elif val in [4, 5]:
        return "Withdrawn / Incomplete"
    elif val == 6:
        return "Purchased"
    else:
        return "Other"

df["action_taken_category"] = df["action_taken"].apply(categorize_action)

# ===============================
# 5️⃣ CLEAN DTI, INCOME, AGE
# ===============================

# DTI as clean text (e.g., "30%-<36%", "Not Provided")
df["debt_to_income_ratio"] = df["debt_to_income_ratio"].astype(str)
df["debt_to_income_ratio"] = df["debt_to_income_ratio"].replace("nan", "Not Provided")

# Income: fill missing
df["income"] = df["income"].fillna("Not Provided")

# Age: keep as text, mark missing as Not Provided
df["applicant_age"] = df["applicant_age"].astype(str).replace("nan", "Not Provided")

# ===============================
# 6️⃣ COUNTY CODE → COUNTY NAME (SAFE)
# ===============================

# Step 1: Fill NaN with 0 so cast won’t fail
df["county_code"] = df["county_code"].fillna(0)

# Step 2: Use nullable Int64 type
df["county_code"] = df["county_code"].astype("Int64")

# Massachusetts FIPS → county name
county_map = {
    25001: "Barnstable",
    25003: "Berkshire",
    25005: "Bristol",
    25007: "Dukes",
    25009: "Essex",
    25011: "Franklin",
    25013: "Hampden",
    25015: "Hampshire",
    25017: "Middlesex",
    25019: "Nantucket",
    25021: "Norfolk",
    25023: "Plymouth",
    25025: "Suffolk",
    25027: "Worcester"
}

df["county_name"] = df["county_code"].map(county_map).fillna("Unknown County")

# ===============================
# 7️⃣ BUILD FINAL CLEAN DATAFRAME
# ===============================
final_cols = [
    "activity_year",
    "state_code",
    "county_name",
    "census_tract",
    "loan_type_label",
    "loan_purpose_label",
    "loan_amount",
    "loan_to_value_ratio",
    "interest_rate",
    "rate_spread",
    "property_value",
    "income",
    "debt_to_income_ratio",
    "applicant_age",
    "derived_dwelling_category",
    "conforming_loan_limit",
    "action_taken_category",
    "action_taken_label"
]

final_df = df[final_cols]

# ===============================
# 8️⃣ SAVE FINAL FILE
# ===============================
output_path = "/content/MA_2023_HMDA_CLEAN_FINAL.csv"
final_df.to_csv(output_path, index=False)

print("🎉 FINAL CLEAN FILE SAVED:", output_path)
print("Rows:", len(final_df))
print("Columns:", len(final_df.columns))
final_df.head()


🎉 FINAL CLEAN FILE SAVED: /content/MA_2023_HMDA_CLEAN_FINAL.csv
Rows: 22554
Columns: 18


,activity_year,state_code,county_name,census_tract,loan_type_label,loan_purpose_label,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,property_value,income,debt_to_income_ratio,applicant_age,derived_dwelling_category,conforming_loan_limit,action_taken_category,action_taken_label
0,2023,MA,Essex,2.500921e+10,Conventional,Refinancing,115000.0,35.48400,7.125,0.622,315000,40.0,46,35-44,Single Family (1-4 Units):Site-Built,C,Approved,Approved - Originated
1,2023,MA,Plymouth,2.502353e+10,Conventional,Refinancing,385000.0,NaN,NaN,NaN,NaN,106.0,Not Provided,45-54,Single Family (1-4 Units):Site-Built,C,Withdrawn / Incomplete,Withdrawn
2,2023,MA,Middlesex,2.501731e+10,FHA-insured,Refinancing,515000.0,97.68900,7.250,1.240,515000,163.0,43,<25,Single Family (1-4 Units):Site-Built,C,Approved,Approved - Originated
3,2023,MA,Middlesex,2.501733e+10,Conventional,Refinancing,495000.0,69.01400,6.325,-0.134,715000,122.0,45,35-44,Single Family (1-4 Units):Site-Built,C,Approved,Approved - Originated
4,2023,MA,Essex,2.500925e+10,Conventional,Refinancing,335000.0,80.96400,7.625,1.391,415000,178.0,30%-<36%,55-64,Single Family (1-4 Units):Site-Built,C,Approved,Approved - Originated


In [ ]:
import pandas as pd
import numpy as np

# 1️⃣ Load base HMDA subset (Massachusetts file you downloaded)
base_path = "/content/loan_purposes_31_state_MA.csv"  # change if your file name is different
df = pd.read_csv(base_path, low_memory=False)

print("Loaded rows:", len(df))
print("Columns:", len(df.columns))


# 2️⃣ Basic filter: keep only MA 2023 (safety, even if already filtered)
if "activity_year" in df.columns:
    df = df[df["activity_year"] == 2023].copy()

if "state_code" in df.columns:
    df = df[df["state_code"] == "MA"].copy()

print("After filtering to 2023 + MA:", len(df), "rows")


# 3️⃣ Decode LOAN TYPE
loan_type_map = {
    1: "Conventional",
    2: "FHA-insured",
    3: "VA-guaranteed",
    4: "USDA (RHS/FSA)"
}
df["loan_type_label"] = df["loan_type"].map(loan_type_map)


# 4️⃣ Decode LOAN PURPOSE
loan_purpose_map = {
    1: "Home Purchase",
    2: "Home Improvement",
    31: "Refinancing",
    32: "Cash-out Refinancing",
    4: "Other Purpose",
    5: "Not Applicable"
}
df["loan_purpose_label"] = df["loan_purpose"].map(loan_purpose_map)


# 5️⃣ Decode ACTION TAKEN → label + category
action_map = {
    1: "Approved - Originated",
    2: "Approved - Not Accepted",
    3: "Denied",
    4: "Withdrawn",
    5: "Incomplete",
    6: "Purchased Loan",
    7: "Preapproval Denied",
    8: "Preapproval Approved - Not Accepted"
}
df["action_taken_label"] = df["action_taken"].map(action_map)

def categorize_action(val):
    if val in [1, 2]:
        return "Approved"
    elif val in [3, 7]:
        return "Denied"
    elif val in [4, 5]:
        return "Withdrawn / Incomplete"
    elif val == 6:
        return "Purchased"
    else:
        return "Other"

df["action_taken_category"] = df["action_taken"].apply(categorize_action)


# 6️⃣ COUNTY CODE → COUNTY NAME (safe)
if "county_code" in df.columns:
    df["county_code"] = df["county_code"].fillna(0)
    df["county_code"] = df["county_code"].astype("Int64")

    county_map = {
        25001: "Barnstable",
        25003: "Berkshire",
        25005: "Bristol",
        25007: "Dukes",
        25009: "Essex",
        25011: "Franklin",
        25013: "Hampden",
        25015: "Hampshire",
        25017: "Middlesex",
        25019: "Nantucket",
        25021: "Norfolk",
        25023: "Plymouth",
        25025: "Suffolk",
        25027: "Worcester"
    }

    df["county_name"] = df["county_code"].map(county_map).fillna("Unknown County")
else:
    df["county_name"] = "Unknown County"


# 7️⃣ INCOME: numeric + buckets
def to_float_safe(x):
    try:
        return float(x)
    except:
        return np.nan

if "income" in df.columns:
    df["income_numeric"] = df["income"].apply(to_float_safe)
else:
    df["income_numeric"] = np.nan

def bucket_income(v):
    if pd.isna(v):
        return "Not Provided"
    if v < 50:
        return "< 50k"
    elif v < 100:
        return "50k–<100k"
    elif v < 150:
        return "100k–<150k"
    else:
        return "150k+"

df["income_bucket"] = df["income_numeric"].apply(bucket_income)


# 8️⃣ LOAN AMOUNT buckets
def bucket_loan_amount(v):
    try:
        v = float(v)
    except:
        return "Not Provided"
    if v < 200000:
        return "< 200k"
    elif v < 400000:
        return "200k–<400k"
    elif v < 600000:
        return "400k–<600k"
    else:
        return "600k+"

df["loan_amount_bucket"] = df["loan_amount"].apply(bucket_loan_amount)


# 9️⃣ DTI: numeric approximation + buckets
def parse_dti(x):
    """
    HMDA DTI can be:
    - numeric string: '36'
    - range string: '30%-<36%'
    - 'Not Available', 'Exempt', etc.
    We will try:
    - if pure number -> float
    - if contains '%' -> take the first number before '%'
    - else -> NaN
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # already numeric?
    try:
        return float(s)
    except:
        pass
    # percentage range like '30%-<36%'
    if "%" in s:
        num_part = s.split("%")[0]
        try:
            return float(num_part)
        except:
            return np.nan
    return np.nan

if "debt_to_income_ratio" in df.columns:
    df["dti_numeric"] = df["debt_to_income_ratio"].apply(parse_dti)
else:
    df["dti_numeric"] = np.nan

def bucket_dti(v):
    if pd.isna(v):
        return "Not Provided"
    if v < 36:
        return "<36%"
    elif v < 43:
        return "36–<43%"
    else:
        return "43%+"

df["dti_bucket"] = df["dti_numeric"].apply(bucket_dti)


# 🔟 RISK FLAGS (for Act 3)

# High LTV: >= 80
df["high_ltv_flag"] = df["loan_to_value_ratio"] >= 80

# High DTI: >= 43
df["high_dti_flag"] = df["dti_numeric"] >= 43

# FHA flag
df["fha_flag"] = df["loan_type_label"] == "FHA-insured"

# Low income flag: < 50k
df["low_income_flag"] = df["income_numeric"] < 50

# Combine risk flags: count how many are True for each loan
risk_cols = ["high_ltv_flag", "high_dti_flag", "fha_flag", "low_income_flag"]
df["risk_flags_count"] = df[risk_cols].sum(axis=1)


# 1️⃣1️⃣ Clean up age & text fields a bit
if "applicant_age" in df.columns:
    df["applicant_age"] = df["applicant_age"].astype(str).replace("nan", "Not Provided")

# ensure derived_dwelling_category exists nicely
if "derived_dwelling_category" not in df.columns:
    df["derived_dwelling_category"] = "Unknown"

# 1️⃣2️⃣ Build final dataset with all key + engineered fields
final_cols = [
    # core identifiers
    "activity_year", "state_code", "county_code", "county_name", "census_tract",

    # loan core
    "loan_type", "loan_type_label",
    "loan_purpose", "loan_purpose_label",
    "loan_amount", "loan_amount_bucket",
    "loan_to_value_ratio", "interest_rate", "rate_spread",
    "property_value",

    # borrower economics
    "income", "income_numeric", "income_bucket",
    "debt_to_income_ratio", "dti_numeric", "dti_bucket",
    "applicant_age",

    # other loan characteristics
    "derived_dwelling_category", "conforming_loan_limit",

    # outcomes
    "action_taken", "action_taken_label", "action_taken_category",

    # risk features
    "high_ltv_flag", "high_dti_flag", "fha_flag",
    "low_income_flag", "risk_flags_count",
]

# keep only columns that actually exist (defensive)
final_cols = [c for c in final_cols if c in df.columns]
final_df = df[final_cols].copy()

output_path = "/content/MA_2023_HMDA_ALL_PAGES_FINAL.csv"
final_df.to_csv(output_path, index=False)

print("✅ Final dataset saved to:", output_path)
print("Rows:", len(final_df))
print("Columns:", len(final_df.columns))
final_df.head()


Loaded rows: 22554
Columns: 99
After filtering to 2023 + MA: 22554 rows


TypeError: '>=' not supported between instances of 'str' and 'int'